In [2]:
import numpy as np
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Вхідний файл, який містить дані
input_file = 'income_data.txt'

# Читання даних
X = []
y = []
count_class1 = 0
count_class2 = 0
max_datapoints = 25000

with open(input_file, 'r') as f:
    for line in f.readlines():
        if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
            break
        if '?' in line:
            continue
        data = line[:-1].split(', ')
        if data[-1] == '<=50K' and count_class1 < max_datapoints:
            X.append(data)
            count_class1 += 1
        if data[-1] == '>50K' and count_class2 < max_datapoints:
            X.append(data)
            count_class2 += 1

# Перетворення на масив numpy
X = np.array(X)

# Перетворення рядкових даних на числові
label_encoder = []
X_encoded = np.empty(X.shape)

for i, item in enumerate(X[0]):
    if item.isdigit():
        X_encoded[:, i] = X[:, i]
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i])

X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)

# Поділ даних на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

# Функція для оцінки моделей
def evaluate_model(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

# Поліноміальне ядро
print("Поліноміальне ядро:")
poly_classifier = OneVsOneClassifier(SVC(kernel='poly', degree=2))
poly_classifier.fit(X_train, y_train)
evaluate_model(poly_classifier, X_test, y_test)

# Гаусове (RBF) ядро
print("Гаусове (RBF) ядро:")
rbf_classifier = OneVsOneClassifier(SVC(kernel='rbf'))
rbf_classifier.fit(X_train, y_train)
evaluate_model(rbf_classifier, X_test, y_test)

# Сигмоїдальне ядро
print("Сигмоїдальне ядро:")
sigmoid_classifier = OneVsOneClassifier(SVC(kernel='sigmoid'))
sigmoid_classifier.fit(X_train, y_train)
evaluate_model(sigmoid_classifier, X_test, y_test)

Поліноміальне ядро:
Accuracy: 0.77
Precision: 0.81
Recall: 0.77
F1 Score: 0.70
Гаусове (RBF) ядро:
Accuracy: 0.78
Precision: 0.83
Recall: 0.78
F1 Score: 0.72
Сигмоїдальне ядро:
Accuracy: 0.60
Precision: 0.61
Recall: 0.60
F1 Score: 0.61
